In [ ]:
#this script will use the graph to creat the dfs graph

In [2]:
import pickle
import json
import geopandas 
import pandas as pd
pd.set_option('max_colwidth', 200)
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors as pltc
from shapely.geometry import Point, Polygon, LineString
import seaborn as sns 
sns.set()
import os
from os import listdir
from os.path import isfile, join
import descartes
import csv
import numpy as np
%matplotlib inline
from random import sample
from math import sin, cos, sqrt, atan2, radians, acos
import numpy as np
import math
import copy
import time

# lakes = geopandas.read_file("Lakes_and_Rivers")
# madison = geopandas.read_file("City_Limit")


ModuleNotFoundError: No module named 'geopandas'

In [4]:
def save_obj(obj, name ):
    with open('obj/'+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name ):
    with open('obj/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [5]:
g=load_obj('graph_final')


In [8]:
def get_stop(node):
    return int(node.split("_")[0])
    
def get_seconds(node):
    return int(node.split("_")[1])
    
# depth-first search through bus system
#
# graph: dict, key=node name, val=edge list
# node: node name
# max_time: last time (in seconds) beyond which we don't search (None if no limit)
# visited: set of nodes visited already (to avoid repeats)
def dfs(graph, node, max_time=None, visited=None, parent_node= None):
    if visited == None:
        visited = set()
    node_time = get_seconds(node)
    if max_time != None and node_time > max_time:
        return visited

    if node in visited:
        return visited
    
    if parent_node:
        trip_id = parent_node['trip_id'] 
    else: 
        trip_id = None
        
    visited.add(node)
    for edge in graph[node]:
            if not trip_id:
                dfs(graph, edge["destination"], max_time, visited, edge)
            else:
                if 'DROP OFF' in edge['full_bus_name']:
                    if edge['trip_id'] == trip_id:
                        dfs(graph, edge["destination"], max_time, visited, edge)
                    else: 
                        continue 
                else:
                    dfs(graph, edge["destination"], max_time, visited, edge)
    return visited

def stops_reachable_percent(start, hours=None):
    limit = None
    if hours != None:
        limit = get_seconds(start) + hours * 3600 # hours to seconds
    reachable = dfs(g, start, limit)
    reachable_stops = {get_stop(node) for node in reachable}
    all_stops = {get_stop(node) for node in g.keys()}
    hour_cover = (len(reachable_stops) / len(all_stops))
    return hour_cover, reachable   

# # where can we get if we start at this place/time, and have no time limit?
# print(stops_reachable_percent('1561_25223'))

# # one hour limit?
# print(stops_reachable_percent('1379_25260', 1))

# # two hour limit?
# print(stops_reachable_percent('1561_25223', 2))


In [9]:
hour_coverage = {}
dfs_output = {}
past_percent = -1
for i,node in enumerate(g):
    hour_coverage[node], dfs_output[node] = stops_reachable_percent(node, 0.5)
    if i*100//len(g) != past_percent:
        past_percent = i*100//len(g)
        if str(0) in str(past_percent):
            print("%d%% done" % past_percent)



0% done
10% done
20% done
30% done
40% done
50% done
60% done
70% done
80% done
90% done


In [10]:
best = max(hour_coverage.values())
print([n for n in g if hour_coverage[n] == best])

['741_26297']


NameError: name 'dfs_output' is not defined

In [19]:
save_obj(dfs_output, 'dfs_output_final')

In [20]:
save_obj(hour_coverage, 'hour_coverage_final')